##### Anisotropia a partir da Magnetização Remanente.
Este script varre valores de Magnetização por Magnetização de Saturação para localizar o valor de Magnetização quando o campo é zero para ambas as metades da curva (conjuntos de dados da "descida" e "subida") do para cada ângulo, descobrindo assim o que chamamos de Magnetização Remanente (Mr)

In [ ]:
"""
CARREANDO BIBLIOTECAS NECESSÁRIAS
"""

import numpy as np #manipulação de arrays e aplicações matemáticas #type: ignore
import matplotlib.pyplot as plt #plot de figuras #type: ignore
import os #manipulação de diretórios/arquivos
from caminhosDiretorios import diretorioDeDadosDesteDispositivo as caminhoDiretorio #variavel que guarda o caminho do diretório dos arquivos de medida
from funcoesNomeDeArquivo import extrairPartesDoNome, extrair_angulo, nomeConcatenadoDaAmostra #de funcoesNomeDeArquivo.py, chamamos aqui funcoes que extraem partes do nome do arquivo -ABQ

"""
CHAMANDO E RECONHECENDO DADOS
"""

caminho_pasta = os.path.join(caminhoDiretorio, "/dadosNormalizados")

# Lista todos os arquivos na pasta
arquivos = os.listdir(caminhoDiretorio)

# Filtra apenas os arquivos .txt
arquivos_txt = [arquivo for arquivo in arquivos if arquivo.endswith('.txt')]

# Ordena os arquivos pelo ângulo
#arquivos_txt.sort(key=extrair_angulo)

"""
OBTENDO VALORES DE M/Ms (na "descida" e na "subida") quando o campo é zero - para cada ângulo
"""

# Lista para armazenar os valores de M/Ms quando o campo é zero para cada ângulo
valores_mm_s = []

# Obtendo os valores de M/Ms quando o campo é zero - tanto na "descida" quanto na "subida" (para cada ângulo)
for arquivo_txt in arquivos_txt:
    # Extrai o ângulo do nome do arquivo
    angulo = extrair_angulo(arquivo_txt)

    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(caminho_pasta, arquivo_txt)
    
    # Carrega os dados do arquivo, ignorando 12 linhas
    dados = np.loadtxt(caminho_arquivo)

    # Extrai os dados de campo e magnetização
    campo = dados[:, 0]
    magnetizacao = dados[:, 1]


    # Encontrando os índices onde o campo é zero durante a descida e durante a subida
    indices_zero_descida = np.where(np.diff(np.signbit(campo)))[0][0]
    indices_zero_subida = np.where(np.diff(np.signbit(campo)))[0][-1] + 1
    
    # Calculando os valores de M/Ms quando o campo é zero durante a descida e durante a subida
    valor_mm_s_descida = magnetizacao[indices_zero_descida]
    valor_mm_s_subida = magnetizacao[indices_zero_subida]

    # Armazenando os valores para este ângulo
    valores_mm_s.append((angulo, valor_mm_s_descida, valor_mm_s_subida))

# Exibindo os valores de M/Ms quando o campo é zero para cada ângulo
print("Valores de M/Ms quando o campo é zero para cada ângulo:")
for angulo, valor_mm_s_descida, valor_mm_s_subida in valores_mm_s:
    print(f'Ângulo {angulo}° (Descida): {valor_mm_s_descida:.2f}')
    print(f'Ângulo {angulo}° (Subida): {valor_mm_s_subida:.2f}')
    
"""
OBTENDO OS VALORES DA MAGNETIZAÇÃO DE REMANÊNCIA PARA CADA ÂNGULO
"""

# Lista para armazenar os valores de magnetização de remanência para cada ângulo
valores_remanencia = []

# Calculando a média em módulo dos valores de M/Ms para cada ângulo
for angulo, valor_mm_s_descida, valor_mm_s_subida in valores_mm_s:
    # Calcula a média em módulo dos valores de descida e subida
    media_descida_subida = (np.abs(valor_mm_s_descida) + np.abs(valor_mm_s_subida)) / 2
    
    # Armazena o valor de magnetização de remanência para este ângulo
    valores_remanencia.append((angulo, media_descida_subida))

# Exibindo os valores de magnetização de remanência para cada ângulo
print("\nValores de magnetização de remanência para cada ângulo:")
for angulo, valor_remanencia in valores_remanencia:
    print(f'Ângulo {angulo}°: Mr = {valor_remanencia:.2f}')


In [ ]:
"""
PLOTANDO A MAGNETIZAÇÃO DE REMANÊNCIA (Mr) PARA CADA ÂNGULO
"""

# Extrai os ângulos e os valores de magnetização de remanência
angulos = [angulo for angulo, _ in valores_remanencia]
valores = [valor for _, valor in valores_remanencia]

# Cria o gráfico de linha
plt.figure(figsize=(10, 6))
plt.plot(angulos, valores, marker='o', color='skyblue', linestyle='-')

# Define os valores de ângulo no eixo x variando de 10 em 10
#plt.xticks(range(min(angulos), max(angulos)+1, 10,))

# Adiciona rótulos e título
plt.xlabel('Ângulo (°)', fontsize=14)
plt.ylabel('Magnetização de Remanência (Mr)', fontsize=14)
plt.title('Magnetização de Remanência (Mr) para Cada Ângulo (amostra: Ta_CoFeB_Ag_x15)', fontsize=16)

# Adiciona grade
plt.grid(True)

# Exibe o gráfico
plt.show()

In [ ]:
############################################
## DESLOCANDO OS DADOS 10° PARA A DIREITA ## (Fazer apenas se necessário)
############################################

# Deslocando os dados 10° para a direita
valores_remanencia_deslocados = [(angulo + 10 if angulo + 10 < 360 else angulo + 10 - 360, valor) for angulo, valor in valores_remanencia]

#########################
## DUPLICANDO OS DADOS ##
#########################

# Criando um dicionário para armazenar valores únicos
valores_remanencia_dict = {angulo: valor for angulo, valor in valores_remanencia_deslocados}

# Atualizando novamente os valores de 0° e 360° após duplicação
if 0 not in valores_remanencia_dict:
    valores_remanencia_dict[0] = valores_remanencia_dict.get(350, valores_remanencia_dict.get(10))
if 360 not in valores_remanencia_dict:
    valores_remanencia_dict[360] = valores_remanencia_dict.get(0, valores_remanencia_dict.get(350))

# Duplicando os dados para 0-360 graus, evitando duplicações para ângulo igual
for angulo, valor_remanencia in list(valores_remanencia_deslocados):
    if angulo > 0 and angulo < 180:
        angulo_duplicado = 360 - angulo
        if angulo_duplicado not in valores_remanencia_dict:
            valores_remanencia_dict[angulo_duplicado] = valor_remanencia

# Ordenando os valores duplicados por ângulo
valores_remanencia_duplicados = sorted(valores_remanencia_dict.items())

# Separando ângulos e valores de remanência
angulos = [x[0] for x in valores_remanencia_duplicados]
remanencias = [x[1] for x in valores_remanencia_duplicados]

# Convertendo ângulos para radianos para plotagem polar
angulos_rad = np.radians(angulos)

########################################
## PLOTANDO OS DADOS EM GRÁFICO POLAR ##
########################################

# Plotando os resultados em coordenadas polares
plt.figure(figsize=(10, 6))
ax = plt.subplot(111, projection='polar')
ax.plot(angulos_rad, remanencias, 'o-', label='Magnetização de Remanência')
ax.set_theta_direction(-1)  # Direção dos ângulos no sentido horário
ax.set_theta_offset(np.pi / 2.0)  # Define 0° no topo
ax.set_xticks(np.radians(np.arange(0, 360, 30)))
ax.set_xticklabels([f'{i}°' for i in range(0, 360, 30)])
plt.title('Anisotropia de Magnetização de Remanência (Mr) para Cada Ângulo (amostra: Ta_CoFeB_Ag_x15)')
#plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#######################
## PLOTANDO OS DADOS ##
#######################

# Plotando os resultados
plt.figure(figsize=(10, 6))
plt.plot(angulos, remanencias, '--o', label='Magnetização de Remanência')
plt.xlabel('Ângulo (graus)')
plt.ylabel('Magnetização de Remanência (M/Ms)')
plt.title('Anisotropia de Magnetização de Remanência (Mr) para Cada Ângulo (amostra: Ta_CoFeB_Ag_x15)')
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))
plt.show()